In [6]:
from azure.storage.blob import BlobServiceClient
import io
import pandas as pd

load_dotenv()  # Load environment variables

# Azure Blob Storage setup
connection_string = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
container_name = os.getenv('AZURE_CONTAINER_NAME')
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Function to read CSV from Azure Blob Storage
def read_csv_from_blob(blob_name,sep=','):
    blob_client = container_client.get_blob_client(blob_name)
    download_stream = blob_client.download_blob()
    return pd.read_csv(io.StringIO(download_stream.content_as_text()),sep=sep)

# Function to write CSV to Azure Blob Storage
def write_csv_to_blob(df, blob_name):
    output = io.StringIO()
    df.to_csv(output, index=False)
    output.seek(0)
    blob_client = container_client.get_blob_client(blob_name)
    blob_client.upload_blob(output.getvalue(), overwrite=True)

# Load data
baby_growth_blob = 'baby_growth_data.csv'
who_data_blob = 'tab_wfa_girls_p_0_13.csv'

try:
    df = read_csv_from_blob(baby_growth_blob)
    df['Date'] = pd.to_datetime(df['Date'])
except:
    df = pd.DataFrame(columns=['Date', 'Age_Days', 'Weight_kg'])

who_data = read_csv_from_blob(who_data_blob,sep=';')
who_data

,Week,L,M,S,P01,P1,P3,P5,P10,P15,P25,P50,P75,P85,P90,P95,P97,P99,P999
0,0,"0,3809","3,2322","0,14171",2,"2,3","2,4","2,5","2,7","2,8","2,9","3,2","3,6","3,7","3,9",4,"4,2","4,4","4,8"
1,1,"0,2671","3,3388","0,146","2,1","2,3","2,5","2,6","2,8","2,9",3,"3,3","3,7","3,9",4,"4,2","4,4","4,6","5,1"
2,2,"0,2304","3,5693","0,14339","2,2","2,5","2,7","2,8",3,"3,1","3,2","3,6","3,9","4,1","4,3","4,5","4,6","4,9","5,4"
3,3,"0,2024","3,8352","0,1406","2,4","2,7","2,9",3,"3,2","3,3","3,5","3,8","4,2","4,4","4,6","4,8",5,"5,3","5,8"
4,4,"0,1789","4,0987","0,13805","2,6","2,9","3,1","3,3","3,4","3,5","3,7","4,1","4,5","4,7","4,9","5,1","5,3","5,6","6,2"
5,5,"0,1582","4,3476","0,13583","2,8","3,1","3,3","3,5","3,6","3,8",4,"4,3","4,8",5,"5,2","5,4","5,6","5,9","6,5"
6,6,"0,1395","4,5793","0,13392",3,"3,3","3,5","3,7","3,8",4,"4,2","4,6",5,"5,3","5,4","5,7","5,9","6,2","6,8"
7,7,"0,1224","4,795","0,13228","3,2","3,5","3,7","3,8",4,"4,2","4,4","4,8","5,2","5,5","5,7","5,9","6,1","6,5","7,1"
8,8,"0,1065","4,9959","0,13087","3,3","3,7","3,9",4,"4,2","4,4","4,6",5,"5,5","5,7","5,9","6,2","6,4","6,7","7,4"
9,9,"0,0918","5,1842","0,12966","3,4","3,8","4,1","4,2","4,4","4,5","4,7","5,2","5,7","5,9","6,1","6,4","6,6",7,"7,7"


In [15]:
pd.to_numeric(who_data['P5'].str.replace(',', '.'), errors='coerce')

0     2.5
1     2.6
2     2.8
3     3.0
4     3.3
5     3.5
6     3.7
7     3.8
8     4.0
9     4.2
10    4.3
11    4.5
12    4.6
13    4.7
Name: P5, dtype: float64

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()  # Load environment variables
print(os.getenv('AZURE_STORAGE_CONNECTION_STRING'))

DefaultEndpointsProtocol=https;AccountName=storageaccountemqx01;AccountKey=JU9C7gPl7aJz8YFQgMa/AA6UTWLOq9CFlBGWs2ZBoJ3Twk6xK/Bhz/JVi7L+Fmdt9PU10PoJ9eOU+AStpjr6Ww==;EndpointSuffix=core.windows.net


In [ ]:

# Process the data to get the percentiles you need
percentiles = {
    '5th': pd.to_numeric(who_data['P5'].str.replace(',', '.'), errors='coerce').tolist(),
    '10th': pd.to_numeric(who_data['P10'].str.replace(',', '.'), errors='coerce').tolist(),
    '50th': pd.to_numeric(who_data['P50'].str.replace(',', '.'), errors='coerce').tolist(),
    '90th': pd.to_numeric(who_data['P90'].str.replace(',', '.'), errors='coerce').tolist(),
    '95th': pd.to_numeric(who_data['P95'].str.replace(',', '.'), errors='coerce').tolist()
}

days = (who_data['Week']*7).tolist()

app.layout = html.Div([
    html.H1("Baby Growth Tracker for Female Infants (WHO Standards)"),
    
    html.Div([
        html.Label("Date of Birth:"),
        dcc.DatePickerSingle(id='dob-picker', date=datetime.now().date() - timedelta(days=14)),
        
        html.Label("Date of Measurement:"),
        dcc.DatePickerSingle(id='date-picker', date=datetime.now().date()),
        
        html.Label("Weight (kg):"),
        dcc.Input(id='weight-input', type='number', placeholder='Enter weight in kg'),
        
        html.Button('Add Record', id='add-button', n_clicks=0)
    ]),
    
    html.Div(id='record-added-message'),
    
    dcc.Graph(id='growth-chart'),
    
    dash_table.DataTable(
        id='data-table',
        columns=[
            {'name': 'Date', 'id': 'Date', 'type': 'datetime'},
            {'name': 'Age (Days)', 'id': 'Age_Days', 'type': 'numeric'},
            {'name': 'Weight (kg)', 'id': 'Weight_kg', 'type': 'numeric'}
        ],
        data=df.to_dict('records'),
        editable=True,
        row_deletable=True
    ),
    
    html.Button('Save Changes', id='save-button', n_clicks=0)
])

@app.callback(
    Output('record-added-message', 'children'),
    Output('growth-chart', 'figure'),
    Output('data-table', 'data'),
    Input('add-button', 'n_clicks'),
    Input('save-button', 'n_clicks'),
    Input('data-table', 'data'),
    State('dob-picker', 'date'),
    State('date-picker', 'date'),
    State('weight-input', 'value')
)
def update_data_and_chart(add_clicks, save_clicks, table_data, dob, date, weight):
    global df
    ctx = dash.callback_context
    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if trigger_id == 'add-button' and dob and date and weight is not None:
        dob = datetime.strptime(dob, '%Y-%m-%d')
        measurement_date = datetime.strptime(date, '%Y-%m-%d')
        age_days = (measurement_date - dob).days
        
        new_record = pd.DataFrame({'Date': [measurement_date], 'Age_Days': [age_days], 'Weight_kg': [weight]})
        df = pd.concat([df, new_record], ignore_index=True)
        df.to_csv(data_file, index=False)
        message = "Record added successfully!"
    elif trigger_id == 'save-button' or trigger_id == 'data-table':
        df = pd.DataFrame(table_data)
        df['Date'] = pd.to_datetime(df['Date'])
        df.to_csv(data_file, index=False)
        message = "Changes saved successfully!"
    else:
        message = "No changes made."
    
    fig = update_chart()
    return message, fig, df.to_dict('records')

def update_chart():
    fig = go.Figure()
    
    # Fill between 5th and 95th percentile with light grey
    fig.add_trace(go.Scatter(
        x=days + days[::-1],
        y=percentiles['95th'] + percentiles['5th'][::-1],
        fill='toself',
        fillcolor='rgba(200,200,200,0.3)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False
    ))

    # Fill between 10th and 90th percentile with darker grey
    fig.add_trace(go.Scatter(
        x=days + days[::-1],
        y=percentiles['90th'] + percentiles['10th'][::-1],
        fill='toself',
        fillcolor='rgba(150,150,150,0.3)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False
    ))
    
    # Add percentile lines
    for percentile, weights in percentiles.items():
        fig.add_trace(go.Scatter(
            x=days,
            y=weights,
            mode='lines',
            name=f'{percentile} Percentile',
            line=dict(color='rgba(0,0,0,0.5)', width=1)
        ))
    
    if not df.empty:
        fig.add_trace(go.Scatter(
            x=df['Age_Days'],
            y=df['Weight_kg'],
            mode='lines+markers',
            line=dict(color='red', width=2),
            marker=dict(size=6),
            name="Baby's Growth"
        ))
    
    fig.update_layout(
        title='Baby Growth Chart for Female Infants (WHO Standards)',
        xaxis_title='Age (days)',
        yaxis_title='Weight (kg)',
        legend=dict(y=0.5, traceorder='reversed', font_size=16),
        hovermode='x unified'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(host='0.0.0.0', port=int(os.environ.get('PORT', 8080)))